<a href="https://colab.research.google.com/github/malinphy/Embedding_calls/blob/main/BERT_MLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd
import random 
import re
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing.sequence import pad_sequences


from tensorflow.keras.layers import Dense, Embedding, Flatten ,MultiHeadAttention,Dropout
from tensorflow.keras import Sequential

import matplotlib.pyplot as plt

In [ ]:
small_sentence = [
                  'my is [MASK]',
                  # 'I live in london'
                  ]
small_tokenizer = Tokenizer()
small_tokenizer.fit_on_texts(small_sentence)
small_tokenizer.word_index

{'is': 2, 'mask': 3, 'my': 1}

In [ ]:
## 1) Sentences will be tokenized using the TextVectorization 
## 2) 12.5 % of the tokens will be replaced with the [MASK] token. Substituded tokens will be 
## chosen randomly. To select the random tokens, random sampling will be chosen 

## 3) segment id will be determined at the second step
## 4) positional embeddings will be directly taken from the article

In [ ]:
max_length =  128  ## 256
corpus_length = 128   ## 30000
embedding_dimension = 128   ## 128
vocab_size = 128  ## 30000
number_heads = 8



In [ ]:
trial_sentences = [
    'my name is john',
    'london is the capital is the england',
    'weather is cold at the northern hemisphere of the world',
    'today, i will go the school'
]


# prediction_sentence = 'my name is [MASK]'
# prediction_tokens = tokenizing_procedure(prediction_sentence, tf_tokenizer)


longest_sentence = []
for i in trial_sentences:
    longest_sentence.append(len(i.split()))
    
print(np.max(longest_sentence))

longest_sentence = np.max(longest_sentence)

10


In [ ]:
tf_tokenizer= Tokenizer()

def tokenizing_procedure(text,tokenizer):
  # this function works as I expected
  tf_tokenizer = tokenizer
  tf_tokens = tf_tokenizer.fit_on_texts(text)

  corpus = tf_tokenizer.word_index
  sequences = tf_tokenizer.texts_to_sequences(text)
  padded_sequences = pad_sequences(sequences)
  corpus['MASK'] = len(corpus)+1

  return padded_sequences, corpus


padded_sequences, corpus = tokenizing_procedure(trial_sentences, tf_tokenizer)




def masking(corpus, padded_sequences,tokenizer):
  masked_corpus = corpus.copy()
  masked_corpus['[MASK]'] = len(corpus)+1
  corpus_tokens = []   #### TOKEN SET WITHOUT THE SPECIAL TOKENS
  for i,j in enumerate(masked_corpus):
      corpus_tokens.append(j)

  subs_tokens=random.sample(corpus_tokens,int(len(corpus_tokens)*0.12))
  subs_tokens

  subs_tokens_ids = [ ]  ## ID OF THE SUBS TOKENS

  for i in subs_tokens:
      subs_tokens_ids.append(tf_tokenizer.word_index[i])

  mask_token_id = 20
  masked_token_id = len(corpus)+1

  for i in range(len(corpus_tokens)):
        if i == (subs_tokens_ids[0]-1   ):
            corpus_tokens[i] = '[MASK]'
    
        if i == (subs_tokens_ids[1]-1):
            corpus_tokens[i] = '[MASK]'

  for i in range(padded_sequences.shape[0]):
        for j in range(padded_sequences.shape[1]):
          for k in range(len(subs_tokens_ids)):
            if padded_sequences[i][j] == (subs_tokens_ids[k]):

                padded_sequences[i][j] = masked_token_id 
            if padded_sequences[i][j] == (subs_tokens_ids[k]):
                
                padded_sequences[i][j] = masked_token_id 


  return padded_sequences, masked_token_id
        
masked_padded_sequence, masked_token_id = masking(corpus, padded_sequences, tf_tokenizer)
masked_padded_sequence



def segment_tokenizer(masked_padded_sequence):
  segment_ids = masked_padded_sequence.copy()
  for i in range(segment_ids.shape[0]):
    for j in range(segment_ids.shape[1]):
      if segment_ids[i][j] != 0:
        segment_ids[i][j] = 1

  return segment_ids

segment_ids = segment_tokenizer(masked_padded_sequence)
# def masked_text_maker(masked_padded_sequences, masked_token_id):

#   for i in range(masked_padded_sequences.shape[0]):
#         for j in range(masked_padded_sequences.shape[1]):
#             if masked_padded_sequences[i][j] != 0:
#                 x = (masked_padded_sequences[i][j]  )
#                 maskeli.append("".join(extended_tokens[x-1]))
                
                
#   maskeli_2 = []
#   for i in (masked_padded_sequences):
#       maskeli_2.append('\n')
#       for j in range(len(i)):
        
#           if (i[j] !=0):
# #            print(i[j])
#               maskeli_2.append("".join(extended_tokens[i[j] -1]))
    
    
#   y= " ".join(maskeli_2)

#   y = y.split('\n')
#   y = np.delete(y,0,0)
  
#   return y

# masked_text_maker(masked_padded_sequence, masked_token_id)

In [ ]:
total_tokens = segment_ids + masked_padded_sequence

total_tokens.shape
padded_sequences

array([[ 0,  0,  0,  0,  0,  0,  3,  4, 22,  5],
       [ 0,  0,  0,  6, 22,  1,  7, 22,  1,  8],
       [ 9, 22, 10, 11,  1, 12, 13, 14,  1, 15],
       [ 0,  0,  0,  0, 16, 17, 22, 19,  1, 20]], dtype=int32)

In [ ]:
# def masking_procedure (text,tok):
    
#     # tf_tokenizer = Tokenizer()
#     tf_tokenizer = tok
#     tf_tokens=tf_tokenizer.fit_on_texts(text)

#     corpus = tf_tokenizer.word_index  ### TOKEN SET WITH TOKEN IDS WITHOUT THE SPECIAL TOKENS 
#     extended_corpus = corpus.copy()   ### TOKEN SET WITH SPECIAL TOKENS

#     extended_corpus['MASK'] = len(corpus)+1

#     corpus_tokens = []   #### TOKEN SET WITHOUT THE SPECIAL TOKENS
#     for i,j in enumerate(corpus):
#         corpus_tokens.append(j)
# #         print(i,j)

        
#     extended_tokens = []
#     for i,j in  enumerate(extended_corpus):
#     #     print(i,j)
#         extended_tokens.append(j)
#     # print('maskelenmis_corpus')
#     # print('-------')
#     subs_tokens=random.sample(corpus_tokens,int(len(corpus_tokens)*0.12)) ## RANDOMLY SELECTED TOKENS TO BE MASKED
# #     print(subs_tokens)

#     sequences = tf_tokenizer.texts_to_sequences(text)
#     padded_sequences= pad_sequences(sequences)
#     padded_sequences

#     substitute_padded_sequences= padded_sequences.copy()
#     actual_padded_sequences = padded_sequences.copy()

#     subs_tokens_ids = [ ]  ## ID OF THE SUBS TOKENS

#     for i in subs_tokens:
#         subs_tokens_ids.append(tf_tokenizer.word_index[i])
    
    
#     mask_token_id = 20
# #     print(subs_tokens_ids)



#     for i in range(len(extended_tokens)):
#         if i == (subs_tokens_ids[0]-1   ):
#             extended_tokens[i] = '[MASK]'
    
#         if i == (subs_tokens_ids[1]-1):
#             extended_tokens[i] = '[MASK]'
        
        
        
#     maskeli = []
#     for i in range(substitute_padded_sequences.shape[0]):
#         for j in range(substitute_padded_sequences.shape[1]):
#             if substitute_padded_sequences[i][j] != 0:
#                 x = (substitute_padded_sequences[i][j]  )
#                 maskeli.append("".join(extended_tokens[x-1]))
                
                
#     maskeli_2 = []
#     for i in (substitute_padded_sequences):
#         maskeli_2.append('\n')
#         for j in range(len(i)):
        
#             if (i[j] !=0):
# #             print(i[j])
#                 maskeli_2.append("".join(extended_tokens[i[j] -1]))
    
    
#     y= " ".join(maskeli_2)

#     y = y.split('\n')
#     y = np.delete(y,0,0)
#     y
     
#     segment_ids = substitute_padded_sequences.copy()

#     for i in range(segment_ids.shape[0]):
#         for j in range(segment_ids.shape[1]):
#             if (segment_ids[i][j] != 0):
#                 segment_ids[i][j] = 1
#     segment_ids = np.array(segment_ids)
    
#     return (y , extended_tokens, segment_ids,substitute_padded_sequences,actual_padded_sequences)

In [ ]:
# tf_tokenizer = Tokenizer()

In [ ]:
# masked_text, masked_tokens,segment_ids , masked_sequence, original_sequences = masking_procedure(trial_sentences,tf_tokenizer)

In [ ]:
# masked_sequence,original_sequences,masked_text

In [ ]:
MAX_LEN = 256
EMBED_DIM = 128



def get_pos_encoding_matrix(max_len, d_emb):
    pos_enc = np.array(
        [
            [pos / np.power(10000, 2 * (j // 2) / d_emb) for j in range(d_emb)]
            if pos != 0
            else np.zeros(d_emb)
            for pos in range(max_len)
        ]
    )
    pos_enc[1:, 0::2] = np.sin(pos_enc[1:, 0::2])  # dim 2i
    pos_enc[1:, 1::2] = np.cos(pos_enc[1:, 1::2])  # dim 2i+1
    return pos_enc


positional_embeddings = get_pos_encoding_matrix(MAX_LEN, EMBED_DIM)

In [ ]:
# pos_enc = get_pos_encoding_matrix(MAX_LEN, EMBED_DIM)
# # pos_enc + segment_ids

# total_encoding = masked_sequence+segment_ids # +pos_enc  (bi sekilde positional encodingi sokmam gerek )

In [ ]:
# total_encoding.shape
# intro_shape = total_encoding.shape
# masked_sequence = masked_sequence.astype(dtype='float32')


In [ ]:
# def encoder_structure (masked_sequence):
#   emb_layer = layers.Embedding(30000, EMBED_DIM,input_length=max_length)(masked_sequence)
#   att_layer = layers.MultiHeadAttention(num_heads=8,key_dim=EMBED_DIM)(emb_layer,emb_layer,emb_layer)
#   drop_layer = layers.Dropout(0.1)(att_layer)
#   norm_layer = layers.LayerNormalization(epsilon = 1e-6)(emb_layer + drop_layer)
#   ff1_layer = layers.Dense(EMBED_DIM , activation='relu')(norm_layer)


#   #bu kisimda bir noktada activasyon fonksiyonu olmadan sadece embedding layeri kadar neurona sahip
#   #baska bir layer var onu da eklemem gerebilir
#   #AYRICA FF1_LAYER'IN CIKIS KISMI SHAPELERINE DIKKAT ET

#   return (ff1_layer)


# decision_model = tf.keras.Sequential([
                                      
#                                       tf.keras.layers.Dense(10,activation = 'softmax'),
#                                       tf.keras.layers.Dense(1)
# ])

# decision_model.compile(
#     loss= 'categorical_crossentropy',
#     optimizer =  'Adam',
#     metrics = ['accuracy']
# )
# decision_input = encoder_structure(masked_sequence)
# decision_model.fit(decision_input, masked_sequence, epochs=1)

# # BU NOKTADAN SONRA SOFTMAX FONKSIYONU KULLANILARAK OLUSTURULAN MODEL TRAIN EDILECEK 
# # INPUT OLARAK ENCODER_STRUCTURE FONKIYONUN 



In [ ]:
# masked_sequence.shape

In [ ]:
# emb_layer = layers.Embedding(30000, EMBED_DIM,input_length=max_length)(masked_sequence)
# att_layer = layers.MultiHeadAttention(num_heads=8,key_dim=EMBED_DIM)(emb_layer,emb_layer,emb_layer)
# drop_layer = layers.Dropout(0.1)(att_layer)
# norm_layer = layers.LayerNormalization(epsilon = 1e-6)(emb_layer + drop_layer)
# ff1_layer = layers.Dense(EMBED_DIM , activation='relu')(norm_layer)


# #### FF NN

# ff_model = tf.keras.Sequential([
#                                 tf.keras.layers.Dense(EMBED_DIM, activation= 'relu'),
#                                 tf.keras.layers.Dense(1)

# ])



# # ff_model.compile(
# #     loss = 'categorical_crossentropy',
# #     optimizer = 'Adam',
# #     metrics = ['accuracy']
# # )

# # ff_model.fit(norm_layer, masked_sequence, epochs =50)
# print('norm_layer_shape',norm_layer.shape)
# print('masked_sequence_shape',masked_sequence.shape)
# print('encoder_structure_out_shape',ff1_layer.shape)

In [ ]:
total_tokens #  INPUT SEQUENCE 
padded_sequences  # LOGITS
total_tokens.shape

(4, 10)

In [ ]:
MAX_LEN = 256
EMBED_DIM = 128



input_layer = keras.Input(shape = (total_tokens.shape[1]))
emb_layer = layers.Embedding(1000, EMBED_DIM)(input_layer)
att_layer = layers.MultiHeadAttention(num_heads=8,key_dim=EMBED_DIM)(emb_layer,emb_layer,emb_layer)
drop_layer = layers.Dropout(0.1)(att_layer)
norm_layer = layers.LayerNormalization(epsilon = 1e-6)(emb_layer + drop_layer)
ff1_layer = layers.Dense(EMBED_DIM , activation='relu')(norm_layer)
output_layer = layers.Dense(1)(ff1_layer)
small_model = keras.Model(input_layer, output_layer)

#### FF NN

ff_model = tf.keras.Sequential([
                                tf.keras.layers.Dense(EMBED_DIM, activation= 'relu'),
                                tf.keras.layers.Dense(1)

])



small_model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'Adam',
    metrics = ['accuracy']
)

small_model.fit(total_tokens, padded_sequences, epochs=3)

Epoch 1/3
1/1 [==============================] - 1s 1s/step - loss: 263.8961 - accuracy: 0.3250
Epoch 2/3
1/1 [==============================] - 0s 17ms/step - loss: 165.8162 - accuracy: 0.3250
Epoch 3/3
1/1 [==============================] - 0s 19ms/step - loss: 148.2724 - accuracy: 0.3250


In [ ]:


prediction_tokens, ikincil = tokenizing_procedure(prediction_sentence, tf_tokenizer)
np.argmax(small_model.predict(prediction_tokens))

0

In [ ]:
prediction_sentence = 'my name is MASK'
tokenizing_procedure(prediction_sentence,tf_tokenizer)

(array([[ 1],
        [ 7],
        [ 0],
        [ 8],
        [ 2],
        [ 1],
        [ 9],
        [ 0],
        [ 6],
        [ 3],
        [ 0],
        [ 1],
        [ 2],
        [ 3],
        [10]], dtype=int32),
 {'MASK': 28,
  'a': 2,
  'at': 19,
  'capital': 15,
  'cold': 18,
  'e': 9,
  'england': 16,
  'go': 26,
  'hemisphere': 21,
  'i': 6,
  'is': 5,
  'john': 13,
  'k': 10,
  'london': 14,
  'm': 1,
  'my': 11,
  'n': 8,
  'name': 12,
  'northern': 20,
  'of': 22,
  's': 3,
  'school': 27,
  'the': 4,
  'today': 24,
  'weather': 17,
  'will': 25,
  'world': 23,
  'y': 7})

In [ ]:
# input_layer = layers.Input(shape = (10,))
# l1_layer = layers.Embedding(corpus_length, 10, input_length =10)(input_layer)
# # l1_layer = layers.Flatten()(l1_layer)
# out_layer  = layers.Dense(10, activation= 'softmax')(l1_layer)
# tf_trial_model  = keras.Model(inputs= input_layer, outputs = out_layer)

# tf_trial_model.compile(
#     loss = 'categorical_crossentropy',
#     optimizer = 'Adam',
#     metrics = ['accuracy']
# )

# tf_trial_model.fit(
#                   #  np.transpose(masked_sequence),
#                    masked_sequence, 
#                    original_sequences, 
#                    epochs = 100)

In [ ]:
cukubi = ['my name is [MASK]',
          'I live in london']

vectorize_layer = TextVectorization(
        max_tokens=100,
        output_mode="int",
        # standardize=custom_standardization,
        output_sequence_length=10,
        
    )
sonc = vectorize_layer.adapt(cukubi)

In [ ]:
vectorize_layer.get_vocabulary()

['', '[UNK]', 'name', 'my', 'mask', 'london', 'live', 'is', 'in', 'i']